In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, matthews_corrcoef, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,learning_curve
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler

In [2]:
model = pickle.load(open('xgb_bank.sav', 'rb'))
df = pd.read_csv('bankM.csv')
df.head()

,age,job,marital,education,default,balance,housing_loan,personal_loan,contact,day,month,duration,campaign,pdays,previous,poutcome,got_loan
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,1
3,47,blue-collar,married,primary,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,1
4,33,services,single,primary,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,1


In [3]:
df = df.rename(columns={'housing_loan': 'housing', 'personal_loan': 'loan'})
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,got_loan
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,1
3,47,blue-collar,married,primary,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,1
4,33,services,single,primary,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,1


In [4]:
ec = LabelEncoder()
df['job'] = ec.fit_transform(df['job'])
df['marital'] = ec.fit_transform(df['marital'])
df['education'] = ec.fit_transform(df['education'])
df['default'] = ec.fit_transform(df['default'])
df['housing'] = ec.fit_transform(df['housing'])
df['loan'] = ec.fit_transform(df['loan'])
df['contact'] = ec.fit_transform(df['contact'])
df['month'] = ec.fit_transform(df['month'])
df['poutcome'] = ec.fit_transform(df['poutcome'])
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,got_loan
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,1
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,1
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,1
3,47,1,1,0,0,1506,1,0,2,5,8,92,1,-1,0,3,1
4,33,7,2,0,0,1,0,0,2,5,8,198,1,-1,0,3,1


In [5]:
data_calc = df[['age','balance','day','duration','campaign','pdays','previous']]
scaler = MinMaxScaler()

scaler.fit(data_calc)
scaled_features = scaler.transform(data_calc)


C:\Users\Rizqa\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [6]:
dataScaled = pd.DataFrame(scaled_features, columns=data_calc.columns)
df.drop(['age','balance','day','duration','campaign','pdays','previous'], axis=1, inplace=True)
df = pd.concat([dataScaled, df.reset_index(drop=True)], axis=1)
df.head()

,age,balance,day,duration,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,month,poutcome,got_loan
0,0.519481,0.092259,0.133333,0.053070,0.0,0.0,0.0,4,1,2,0,1,0,2,8,3,1
1,0.337662,0.073067,0.133333,0.030704,0.0,0.0,0.0,9,2,1,0,1,0,2,8,3,1
2,0.194805,0.072822,0.133333,0.015453,0.0,0.0,0.0,2,1,1,0,1,1,2,8,3,1
3,0.376623,0.086476,0.133333,0.018707,0.0,0.0,0.0,1,1,0,0,1,0,2,8,3,1
4,0.194805,0.072812,0.133333,0.040260,0.0,0.0,0.0,7,2,0,0,0,0,2,8,3,1


In [12]:
gotdict={1:0,2:1}
df['got_loan'] = df.got_loan.map(gotdict)

In [21]:
# data = df[df['got_loan']==0]
data = df
data.head()

,age,balance,day,duration,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,month,poutcome,got_loan
0,0.519481,0.092259,0.133333,0.053070,0.0,0.0,0.0,4,1,2,0,1,0,2,8,3,0
1,0.337662,0.073067,0.133333,0.030704,0.0,0.0,0.0,9,2,1,0,1,0,2,8,3,0
2,0.194805,0.072822,0.133333,0.015453,0.0,0.0,0.0,2,1,1,0,1,1,2,8,3,0
3,0.376623,0.086476,0.133333,0.018707,0.0,0.0,0.0,1,1,0,0,1,0,2,8,3,0
4,0.194805,0.072812,0.133333,0.040260,0.0,0.0,0.0,7,2,0,0,0,0,2,8,3,0


In [22]:
X_test = data.drop(['got_loan'], axis=1)
y_test = data['got_loan']

In [23]:
result = model.score(X_test, y_test)

In [24]:
print(result)

0.8822410475326801


In [25]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=nan, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=101, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [ ]:
model.score()